In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import TextIOWrapper
from zipfile import ZipFile
import io

from matplotlib.colors import LinearSegmentedColormap

from numpy import array
from numpy import argmax

from sklearn import tree
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


# Files were previously uploaded from data cleaning onto Github repository.
# Just need to pull them up with the following:

feat_url = "https://raw.githubusercontent.com/jasmultani5391/Census-Data/master/featDF.csv"
download1 = requests.get(feat_url).content

complete_url = "https://raw.githubusercontent.com/jasmultani5391/Census-Data/master/completeDF.csv"
download2 = requests.get(complete_url).content

# Read the downloaded content and turn it into a pandas dataframe
featDF = pd.read_csv(io.StringIO(download1.decode('utf-8')))
completeDF = pd.read_csv(io.StringIO(download2.decode('utf-8')))

#print(featDF.head(4))
#print(completeDF.head(4))

print('How would I iterate through this? Would I even need to? Still need to find the measurements')


How would I iterate through this? Would I even need to? Still need to find the measurements


In [2]:
labelDF = featDF['salary_label']
featDF = featDF.drop(['salary_label'],
                     axis=1
                     )

In [3]:
################# STEP 7: LOGISTIC REGRESSION CLASSIFIER #################
# Let's now train the data on logistic regression. Split up the dataset
# between trained and tested data by making the test_size=0.2.


trnlogdata, tstlogdata, trnloglbl, tstloglbl = train_test_split(featDF,
                                                                labelDF,
                                                                test_size=0.2,
                                                                random_state=0
                                                               )

# Transform the data using the StandardScaler() class. The Logistic Regression
# classifier needs the data to be in a standardized shape.
scaler = StandardScaler()
trnlogdata = scaler.fit_transform(trnlogdata)
tstlogdata = scaler.transform(tstlogdata)

# Create and train the model on the LogisticRegression class.
lr_classifier = LogisticRegression(random_state=0,
                                   fit_intercept=True,
                                  ).fit(trnlogdata, trnloglbl)

predict = lr_classifier.predict(trnlogdata[:3, :]) #Was the prediction label 1 or 0
print(predict)
#likelihood that the prediction is a 1
predict_est = lr_classifier.predict_proba(trnlogdata)[:, :]
predict_est1 = np.reshape(predict_est[:100,1], (-1,1))


# Score the model on the train data.
train_logreg_model = lr_classifier.score(trnlogdata, trnloglbl)
#print('Train score is ' + str(round(train_logreg_model*100, 2)))

# Score the model on the test data.
test_logreg_model = lr_classifier.score(tstlogdata, tstloglbl)
#print('Test score is ' + str(round(test_logreg_model*100, 2)))

clf = lr_classifier.sparsify()
#print(clf)

[0 0 0]


In [4]:
# Train model with best k (where F1 is highest)
lr_classifier.fit(trnlogdata, np.ravel(trnloglbl))


# Score based on train dataset.
train_lrscore = lr_classifier.score(trnlogdata, trnloglbl)
print('Train Accuracy Score %: ' + str(round(train_lrscore*100, 4)))

predict_trainlabels = lr_classifier.predict(trnlogdata)
trainprecision = precision_score(trnloglbl, predict_trainlabels, average='binary')
print('Train Precision %: ' + str(round(trainprecision*100, 4)))
            
trainrecall = recall_score(trnloglbl, predict_trainlabels, average='binary')
print('Train Recall %: ' + str(round(trainrecall*100, 4)))

trainf1 = f1_score(trnloglbl, predict_trainlabels, average='binary')
print('Train F1 %: ' + str(round(trainf1*100, 4)))


print('\n\n')
# Score based on test dataset.
test_lrscore = lr_classifier.score(tstlogdata, tstloglbl)
print('Test Accuracy Score %: ' + str(round(test_lrscore*100, 4)))

predict_testlabels = lr_classifier.predict(tstlogdata)
testprecision = precision_score(tstloglbl, predict_testlabels, average='binary')
print('Test Precision %: ' + str(round(testprecision*100, 4)))
            
testrecall = recall_score(tstloglbl, predict_testlabels, average='binary')
print('Test Recall %: ' + str(round(testrecall*100, 4)))

testf1 = f1_score(tstloglbl, predict_testlabels, average='binary')
print('Test F1 %: ' + str(round(testf1*100, 4)))

Train Accuracy Score %: 85.3501
Train Precision %: 73.1667
Train Recall %: 58.7693
Train F1 %: 65.1825



Test Accuracy Score %: 85.0476
Test Precision %: 75.0784
Test Recall %: 59.3556
Test F1 %: 66.2976


In [5]:
train_predict = lr_classifier.predict(trnlogdata)
confusion_matrix = confusion_matrix(trnloglbl, train_predict)
tn, fp, fn, tp = confusion_matrix.flatten()
print(tn, fp, fn, tp )
report_train = classification_report(trnloglbl, train_predict)
print(report_train)

9330 655 1253 1786
              precision    recall  f1-score   support

           0       0.88      0.93      0.91      9985
           1       0.73      0.59      0.65      3039

    accuracy                           0.85     13024
   macro avg       0.81      0.76      0.78     13024
weighted avg       0.85      0.85      0.85     13024

